<a href="https://colab.research.google.com/github/bettercodepaul/data-wrangling-praktikum/blob/master/dashBootstrapNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dash Einführung
Dash ist ein Framework zur Erstellung von Webapps für die Datenvisualisierung, entwickelt von Plotly. Dash zielt darauf ab, dass der User so weit wie möglich in Python arbeiten kann und eignet sich gut dafür, mit Polars zusammen verwendet zu werden.

In diesem Notebook werden die Grundlagen von Dash anhand von Beispielen vorgestellt. Die offizielle Dokumentation befindet sich hier: 

https://dash.plotly.com/

*Anmerkung: Damit Dash in der Colab-Umgebung richtig funktioniert, sollten JavaScript und Cookies im Browser erlaubt sein.*

## Installation + Vorbereitung

In [ ]:
import urllib.request
import os.path

In [ ]:
REQUIREMENTS_URL = "https://github.com/bettercodepaul/data-wrangling-praktikum/raw/master/requirements.txt"
urllib.request.urlretrieve(REQUIREMENTS_URL, os.path.basename(REQUIREMENTS_URL))

In [ ]:
# nicht vergessen, dass die Laufzeitumgebung ggf. neu gestartet werden muss
!pip install -qr requirements.txt

In [ ]:
# Daten herunterladen
DATA_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/spotify-charts-2017-2021-global-top200.csv.gz"
LOCAL_DATA_FILE_NAME = os.path.basename(DATA_URL)
urllib.request.urlretrieve(DATA_URL, LOCAL_DATA_FILE_NAME)
GENRES_DATA_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/track-genres.parquet"
LOCAL_GENRES_DATA_FILE_NAME = os.path.basename(GENRES_DATA_URL)
urllib.request.urlretrieve(GENRES_DATA_URL, LOCAL_GENRES_DATA_FILE_NAME)
BIG_DATA_URL = "https://github.com/bettercodepaul/data2day_2023_polars/releases/download/data-parquet/spotify-charts-2017-2021.parquet"
LOCAL_BIG_DATA_FILE_NAME = os.path.basename(BIG_DATA_URL)
urllib.request.urlretrieve(BIG_DATA_URL, LOCAL_BIG_DATA_FILE_NAME)

In [ ]:
# Hilfsfunktionen herunterladen
EXERCISES_URL = "https://github.com/bettercodepaul/data-wrangling-praktikum/raw/master/utils_exercises.py"
urllib.request.urlretrieve(EXERCISES_URL, os.path.basename(EXERCISES_URL))

In [ ]:
from utils_exercises import *

In [ ]:
import polars as pl

In [ ]:
# bis zu 60 Zeichen pro Spalte ausgeben und Fließkommazahlen nicht abkürzen
pl.Config(fmt_str_lengths=60, fmt_float="full")

In [ ]:
track_genres = pl.read_parquet(LOCAL_GENRES_DATA_FILE_NAME)
df = df = (
    pl.read_csv(LOCAL_DATA_FILE_NAME, try_parse_dates=True)
    .join(track_genres, on="url", how="left")
)
df.head(2) # die ersten 2 Zeilen ausgeben



In [ ]:
region_df = (
    pl.read_parquet(LOCAL_BIG_DATA_FILE_NAME)
    .with_columns(pl.col("streams").cast(pl.Int64))
    .filter(pl.col("chart").eq("top200") & pl.col("region").ne("Global"))
)
# only keep top 25 regions
region_df = (
    region_df.join(
        region_df.group_by("region").agg(pl.sum("streams")).top_k(25, by="streams"),
        on="region", how="semi"
    )
)
region_df.head(2) # die ersten 2 Zeilen ausgeben

In [ ]:
from dash import dcc, html, Dash, _dash_renderer, Input, Output
import plotly.express as px
import dash_mantine_components as dmc
# React-Version auf 18.2.0 setzen (wird von Dash Mantine Components benötigt)
_dash_renderer._set_react_version("18.2.0")

## Statische Apps

Mit Dash können sehr aufwändige Oberflächen mit komplexen Diagrammen, Auswahl- und Filterelementen (z.B. Dropdown-Menüs, Checkboxen, Slider, etc.) und Layouts erstellt werden.

Normalerweise wird Dash als normales Pythonskript ausgeführt, der Zugriff auf die erzeugte Visualisierung erfolgt über den Browser. Innerhalb dieses Notebooks/der Colab-Umgebung greifen wir aber auf die Inline-Darstellung zurück.

In der Praxis könntet ihr eine Dash-App sehr einfach in einen Container packen und zum Beispiel in einem Kubernetes-Cluster oder Cloud-Service laufen lassen.

Für den Anfang erstmal eine minimale Dash-Anwendung:

In [ ]:
app = Dash(__name__)

# Das Layout wird ähnlich wie in HTML festgelegt
app.layout = html.Div(children=[
    html.H1(children="Hallo"),
    html.Div(children="Das ist Deine erste Dash-App.")
])

# Die Dash-App wird im Notebook selber angezeigt (jupyter_mode="inline")
app.run(jupyter_mode="inline", jupyter_height=150)

Für das Layout verwenden wir in diesem Notebook **Dash Mantine Components** (https://www.dash-mantine-components.com). Dies ändert nichts an den Konzepten von Dash, macht den Layout-Code bei etwas komplexeren Beispielen aber deutlich übersichtlicher und bringt mehr fertige und konsistent gestylte UI-Komponenten mit.

In [ ]:
app = Dash(external_stylesheets=dmc.styles.ALL)

# Wir müssen alle Komponenten in einem MantineProvider einschließen
app.layout = dmc.MantineProvider([
     dmc.Title("Hallo", order=1),
     dmc.Text("Das ist Deine erste Dash-App mit Mantine.")
])

# Die Dash-App wird im Notebook selber angezeigt (jupyter_mode="inline")
app.run(jupyter_mode="inline", jupyter_height=150)

Um Diagramme für unsere Daten erstellen zu können verwenden wir das `plotly.express` Modul, oben importiert als `px`. Es ermöglicht die Erstellung interaktiver Diagramme im Browser (das oft benutzte Matplotlib dagegen erzeugt nur statische Diagramme): https://plotly.com/python/plotly-express/

In [ ]:
px.line(df.group_by("date").agg(pl.col("streams").sum()).sort("date"), x="date", y="streams", height=300)

Mit einem `dcc.Graph` können wir das Diagramm auch in einer Dash-App anzeigen.

In [ ]:
app = Dash(external_stylesheets=dmc.styles.ALL)

app.layout = dmc.MantineProvider([
     dmc.Title("Dash-App mit Diagramm", order=5),
     # Diagramm wird mit dem Attribut figure übergeben
     dcc.Graph(id="streams-chart", figure=px.line(df.group_by("date").agg(pl.col("streams").sum()).sort("date"), x="date", y="streams", height=300))
])

app.run(jupyter_mode="inline", jupyter_height=350)

### Übung

Jetzt zu den Aufgaben für Dich:



In [ ]:
q1.question()

In [ ]:
q1_df = ...

In [ ]:
q1.check(q1_df)

In [ ]:
q2.question()

In [ ]:
q2_fig = ...
q2_fig

In [ ]:
q2.check(q2_fig)

## Interaktive Apps mit Callbacks
Ein höherer Grad an Interaktion wird in Dash über Callbacks erreicht. Mit Callbacks kann auf Userinputs reagiert werden, um zum Beispiel bei der Auswahl in einer Dropdown-Komponente eine andere UI-Komponenten wie ein Diagramme anzupassen. Auch hierzu wieder ein Minimalbeispiel:

In [ ]:
app = Dash(external_stylesheets=dmc.styles.ALL)

# Für Callbacks sind die ids der Komponenten wichtig
app.layout = dmc.MantineProvider([
     dmc.TextInput(id="my-input", value=None, label="Deine Eingabe", placeholder="Hier beliebigen Text eingeben..."),
     dmc.TextInput(id="my-output", value=None, label="Meine Ausgabe", disabled=True)
])


# Der Callback wird vom Client aufgerufen, sobald sich eine Property des Inputs ändert
# Die Rückgabe des Callbacks wird vom Client in die Property des Outputs geschrieben
@app.callback(
    Output(component_id="my-output", component_property="value"),
    Input(component_id="my-input", component_property="value")
)
def update_output_div(input_value):
    output_value = input_value
    return output_value

app.run(jupyter_mode="inline", jupyter_height=150)

Callbacks werden in Dash mit `@app.callback` und den Output- und Inputparametern annotiert. Die Annotation verknüpft die Callbackfunktion über ID- und Property-Werte mit den entsprechenden Elementen, die im Layout spezifiziert sind. Alle Callbacks werden beim Start der Anwendung einmal ausgeführt, um Elemente initial einzurichten.

<pre>
# Für Callbacks sind die ids der Komponenten wichtig
app.layout = dmc.MantineProvider([
     dmc.TextInput(id=<span style="color:#4169E1">"my-input"</span>, <span style="color:#4169E1">value</span>=<span style="color:white; background-color:#4169E1">None</span>, label="Deine Eingabe", placeholder="Hier beliebigen Text eingeben..."),
     dmc.TextInput(id=<span style="color:#C04000">"my-output"</span>, <span style="color:#C04000">value</span>=<span style="color:white; background-color:#C04000">None</span>, label="Meine Ausgabe", disabled=True)
])


# Der Callback wird vom Client aufgerufen, sobald sich eine Property des Inputs ändert
# Die Rückgabe des Callbacks wird vom Client in die Property des Outputs geschrieben
@app.callback(
    Output(component_id=<span style="color:#C04000">"my-output"</span>, component_property=<span style="color:#C04000">"value"</span>),
    Input(component_id=<span style="color:#4169E1">"my-input"</span>, component_property=<span style="color:#4169E1">"value"</span>)
)
def update_output_div(<span style="color:white; background-color:#4169E1">input_value</span>):
    output_value = input_value
    return <span style="color:white; background-color:#C04000">output_value</span>
</pre>

Über Properties werden die Attribute der Layout-Elemente spezifiziert, die für die Callback-Funktion relevant sind. Im Output können auch mehrere Attribute gleichzeitig verändert werden, ein Beispiel hierzu kommt am Ende des Notebooks.

Mithilfe von Callbacks bauen wir nun eine einfache Filtermöglichkeit ein:

In [ ]:
all_artists = df.select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()

app = Dash(external_stylesheets=dmc.styles.ALL)

def get_streams_chart(artist):
    filter_expr = pl.lit(True) if artist is None else pl.col("artist").str.contains(artist)
    data = df.filter(filter_expr).group_by("date").agg(pl.col("streams").sum()).sort("date")
    return px.line(data, x="date", y="streams", height=300, title=f"Daily Streams for {artist or 'all artists'}")

app.layout = dmc.MantineProvider([
    dmc.Title("Spotify Explorer", order=3, mb=20),
    dmc.Select(id="artist-select", label="Artist", placeholder="Select one", data=all_artists, w="400", searchable=True, mb=10),
    dcc.Graph(id="streams-chart", figure=get_streams_chart(None))
])

@app.callback(
    Output(component_id="streams-chart", component_property="figure"),
    Input(component_id="artist-select", component_property="value")
)
def update_streams_per_month(selected_artist):
    return get_streams_chart(selected_artist)

app.run(jupyter_mode="inline", jupyter_height=500)

### Mehrere Inputs

Wir möchten zusätzlich nach dem Titel filtern können.

Implementierung in der Dash-App mit zusätzlichen Input-Komponenten.

In [ ]:
all_artists = df.select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
all_titles = df.select(pl.col("title").unique()).get_column("title").sort().to_list()

app = Dash(external_stylesheets=dmc.styles.ALL)

def get_streams_chart(artist, title):
    filter_expr = pl.lit(True) if artist is None else pl.col("artist").str.contains(artist)
    if title is not None:
        filter_expr = filter_expr & pl.col("title").eq(title)
    data = df.filter(filter_expr).group_by("date").agg(pl.col("streams").sum()).sort("date")
    return px.line(data, x="date", y="streams", height=300, title=f"Daily Streams for {artist or 'all artists'} - {title or 'all titles'}")

app.layout = dmc.MantineProvider([
    dmc.Title("Spotify Explorer", order=3, mb=20),
    dmc.Group([
        dmc.Select(id="artist-select", label="Artist", placeholder="Select one", data=all_artists, searchable=True),
        dmc.Select(id="title-select", label="Title", placeholder="Select one", data=all_titles, searchable=True)
    ], grow=True, mb=10),
    dcc.Graph(id="streams-chart", figure=get_streams_chart(None, None))
])

@app.callback(
    Output(component_id="streams-chart", component_property="figure"),
    Input(component_id="artist-select", component_property="value"),
    Input(component_id="title-select", component_property="value")
)
def update_streams_per_month(selected_artist, selected_title):
    return get_streams_chart(selected_artist, selected_title)

app.run(jupyter_mode="inline", jupyter_height=500)

Wir können auch die angezeigten Optionen in Abhängigkeit der gewählten Optionen einschränken.

In [ ]:
all_artists = df.select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
all_titles = df.select(pl.col("title").unique()).get_column("title").sort().to_list()

app = Dash(external_stylesheets=dmc.styles.ALL)

def get_streams_chart(artist, title):
    filter_expr = pl.lit(True) if artist is None else pl.col("artist").str.contains(artist)
    if title is not None:
        filter_expr = filter_expr & pl.col("title").eq(title)
    data = df.filter(filter_expr).group_by("date").agg(pl.col("streams").sum()).sort("date")
    return px.line(data, x="date", y="streams", height=300, title=f"Daily Streams for {artist or 'all artists'} - {title or 'all titles'}")

app.layout = dmc.MantineProvider([
    dmc.Title("Spotify Explorer", order=3, mb=20),
    dmc.Group([
        dmc.Select(id="artist-select", label="Artist", placeholder="Select one", data=all_artists, searchable=True),
        dmc.Select(id="title-select", label="Title", placeholder="Select one", data=all_titles, searchable=True)
    ], grow=True, mb=10),
    dcc.Graph(id="streams-chart", figure=get_streams_chart(None, None))
])

@app.callback(
    Output(component_id="streams-chart", component_property="figure"),
    Output(component_id="artist-select", component_property="data"), # mögliche Künstler für die aktuelle Filterung
    Output(component_id="title-select", component_property="data"), # mögliche Titel für die aktuelle Filterung
    Input(component_id="artist-select", component_property="value"),
    Input(component_id="title-select", component_property="value")
)
def update_streams_per_month(selected_artist, selected_title):
    # mögliche Künstler für die aktuelle Filterung ermitteln
    if selected_artist is not None:
        possible_titles = df.filter(pl.col("artist").str.contains(selected_artist)).select(pl.col("title").unique()).get_column("title").sort().to_list()
    else:
        possible_titles = all_titles
    # mögliche Titel für die aktuelle Filterung ermitteln
    if selected_title is not None:
        possible_artists = df.filter(pl.col("title").eq(selected_title)).select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
    else:
        possible_artists = all_artists
    return get_streams_chart(selected_artist, selected_title), possible_artists, possible_titles

app.run(jupyter_mode="inline", jupyter_height=500)

### Übung

In [ ]:
q3.question()

In [ ]:
# Platz für deine Lösung

In [ ]:
q3.hint()

## IFrame

Damit wir die Lieder auch anhören können, bauen wir einen IFrame mit dem Spotify Player ein, der immer den am meisten gestreamten Song abspielt.

In [ ]:
all_artists = df.select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
all_titles = df.select(pl.col("title").unique()).get_column("title").sort().to_list()

app = Dash(external_stylesheets=dmc.styles.ALL)

def filter_data(artist, title, rank_1_only):
    filter_expr = pl.lit(True) if artist is None else pl.col("artist").str.contains(artist)
    if title is not None:
        filter_expr = filter_expr & pl.col("title").eq(title)
    if rank_1_only:
        # alle Lieder berücksichtigen, die mindestens einmal auf Platz 1 waren
        filter_expr = filter_expr & pl.col("rank").min().over("artist", "title").eq(1)
    return df.filter(filter_expr)


def get_streams_chart(filtered_data, title):
    data = filtered_data.group_by("date").agg(pl.col("streams").sum()).sort("date")
    return px.line(data, x="date", y="streams", height=300, title=title)


def get_top_song_player(filtered_data):
    top_song = filtered_data.group_by("url").agg(pl.col("streams").sum()).top_k(1, by="streams")
    return play_song(top_song)


app.layout = dmc.MantineProvider([
    dmc.Title("Spotify Explorer", order=3, mb=20),
    dmc.Group([
        dmc.Select(id="artist-select", label="Artist", placeholder="Select one", data=all_artists, searchable=True),
        dmc.Select(id="title-select", label="Title", placeholder="Select one", data=all_titles, searchable=True),
        # Schalter für die Option, nur Lieder anzuzeigen, die mindestens einmal auf Platz 1 waren
        dmc.Switch(id="rank-1-switch", label="#1 hits only", checked=False)
    ], grow=True, mb=10),
    dcc.Graph(id="streams-chart", figure=get_streams_chart(df, "Daily Streams for all artists - 'all titles'")),
    dmc.Title("Most streamed title", order=4, mb=10),
    html.Iframe(id="player-iframe", src=None, width="100%", height="80", style={"border": "none"})
])

@app.callback(
    Output(component_id="streams-chart", component_property="figure"),
    Output(component_id="artist-select", component_property="data"),
    Output(component_id="title-select", component_property="data"),
    Output(component_id="player-iframe", component_property="src"),
    Input(component_id="artist-select", component_property="value"),
    Input(component_id="title-select", component_property="value"),
    Input(component_id="rank-1-switch", component_property="checked")
)
def update(selected_artist, selected_title, rank_1_only):
    filter_expr = pl.lit(True) if not rank_1_only else pl.col("rank").eq(1)
    if selected_artist is not None:
        possible_titles = df.filter(filter_expr & pl.col("artist").str.contains(selected_artist)).select(pl.col("title").unique()).get_column("title").sort().to_list()
    else:
        possible_titles = df.filter(filter_expr).select(pl.col("title").unique()).get_column("title").sort().to_list()
    if selected_title is not None:
        possible_artists = df.filter(filter_expr & pl.col("title").eq(selected_title)).select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
    else:
        possible_artists = df.filter(filter_expr).select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
    filtered_data = filter_data(selected_artist, selected_title, rank_1_only)
    streams_chart_figure = get_streams_chart(filtered_data, f"Daily Streams for {selected_artist or 'all artists'} - {selected_title or 'all titles'}")
    player = get_top_song_player(filtered_data)    
    return streams_chart_figure, possible_artists, possible_titles, player.src

app.run(jupyter_mode="inline", jupyter_height=550)

## Tabellen

Wir können auch eine Tabelle einbauen, in der eine Übersicht zu den Liedern gezeigt wird. Wir nutzen dafür die sehr mächtige AgGrid-Bibliothek, die es auch für Dash gibt: https://dash.plotly.com/dash-ag-grid

In [ ]:
import dash_ag_grid as dag

In [ ]:
all_artists = df.select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
all_titles = df.select(pl.col("title").unique()).get_column("title").sort().to_list()

app = Dash(external_stylesheets=dmc.styles.ALL)

def filter_data(artist, title, rank_1_only):
    filter_expr = pl.lit(True) if artist is None else pl.col("artist").str.contains(artist)
    if title is not None:
        filter_expr = filter_expr & pl.col("title").eq(title)
    if rank_1_only:
        # alle Lieder berücksichtigen, die mindestens einmal auf Platz 1 waren
        filter_expr = filter_expr & pl.col("rank").min().over("artist", "title").eq(1)
    return df.filter(filter_expr)


def get_streams_chart(filtered_data, title):
    data = filtered_data.group_by("date").agg(pl.col("streams").sum()).sort("date")
    return px.line(data, x="date", y="streams", height=300, title=title)


def get_top_song_player(filtered_data):
    top_song = filtered_data.group_by("url").agg(pl.col("streams").sum()).top_k(1, by="streams")
    return play_song(top_song)

def get_grid_data(filtered_data):
    grid_data = (
        filtered_data
        .group_by("artist", "title")
        .agg(
            pl.col("streams").sum(),
            pl.col("rank").min().alias("highest rank")
        )
        .top_k(k=10, by="streams")
        .sort("streams", descending=True)
    )
    return grid_data 

def get_grid_component(grid_data):
    column_defs = [{"field": column_name} for column_name in grid_data.columns]
    return dag.AgGrid(
        id="top-songs-grid",
        rowData=grid_data.to_dicts(),
        columnDefs=column_defs,
        columnSize="autoSize",
    )


app.layout = dmc.MantineProvider([
    dmc.Title("Spotify Explorer", order=3, mb=20),
    dmc.Group([
        dmc.Select(id="artist-select", label="Artist", placeholder="Select one", data=all_artists, searchable=True),
        dmc.Select(id="title-select", label="Title", placeholder="Select one", data=all_titles, searchable=True),
        # Schalter für die Option, nur Lieder anzuzeigen, die mindestens einmal auf Platz 1 waren
        dmc.Switch(id="rank-1-switch", label="#1 hits only", checked=False)
    ], grow=True, mb=10),
    dcc.Graph(id="streams-chart", figure=get_streams_chart(df, "Daily Streams for all artists - 'all titles'")),
    dmc.Title("Most streamed titles", order=4, mb=10),
    html.Iframe(id="player-iframe", src=None, width="100%", height="80", style={"border": "none"}),
    dmc.Space(h=10),
    get_grid_component(get_grid_data(df))
])

@app.callback(
    Output(component_id="streams-chart", component_property="figure"),
    Output(component_id="artist-select", component_property="data"),
    Output(component_id="title-select", component_property="data"),
    Output(component_id="player-iframe", component_property="src"),
    Output(component_id="top-songs-grid", component_property="rowData"),
    Input(component_id="artist-select", component_property="value"),
    Input(component_id="title-select", component_property="value"),
    Input(component_id="rank-1-switch", component_property="checked")
)
def update(selected_artist, selected_title, rank_1_only):
    filter_expr = pl.lit(True) if not rank_1_only else pl.col("rank").eq(1)
    if selected_artist is not None:
        possible_titles = df.filter(filter_expr & pl.col("artist").str.contains(selected_artist)).select(pl.col("title").unique()).get_column("title").sort().to_list()
    else:
        possible_titles = df.filter(filter_expr).select(pl.col("title").unique()).get_column("title").sort().to_list()
    if selected_title is not None:
        possible_artists = df.filter(filter_expr & pl.col("title").eq(selected_title)).select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
    else:
        possible_artists = df.filter(filter_expr).select(pl.col("artist").str.split(", ")).get_column("artist").explode().unique().sort().to_list()
    filtered_data = filter_data(selected_artist, selected_title, rank_1_only)
    streams_chart_figure = get_streams_chart(filtered_data, f"Daily Streams for {selected_artist or 'all artists'} - {selected_title or 'all titles'}")
    player = get_top_song_player(filtered_data)    
    grid_data = get_grid_data(filtered_data)
    return streams_chart_figure, possible_artists, possible_titles, player.src, grid_data.to_dicts()

app.run(jupyter_mode="inline", jupyter_height=1000)

# Abschließende freie Übung

Du hast jetzt verschiedene Optionen:

- erweitere den Spotify Explorer nach deinen Vorstellungen, zum Beispiel...
    - Optionen zur Anzeige und Filterung nach Genres
    - zusätzliche Visualisierungen (Verlauf der Platzierung (rank), ...)
    - zusätzliche Filter nach Zeit (Jahr, Tag, Wochentag, Monat, ...)
    - zusätzliche Informationen in der Tabelle (Anzahl der Tage auf Nr. 1, in den Top 10, ...)
    - lasse dich von den Mantine Komponenten inspieren (https://www.dash-mantine-components.com)
- konzipiere eine Dash-Applikation, die euch im Projekt helfen würde
- installiere eine Python-Umgebung auf deinem lokalen Rechner (siehe unten)
- arbeite weiter an den Übungen zu Polars, ggf. auch mit dem dritten Notebook, dass wir nicht behandelt haben
    - [Teil 1](https://colab.research.google.com/github/bettercodepaul/data2day_2023_polars/blob/main/data2day_2023_Polars_Teil_1.ipynb) - Laden, Select, Filtern & Sortieren
    - [Teil 2](https://colab.research.google.com/github/bettercodepaul/data2day_2023_polars/blob/main/data2day_2023_Polars_Teil_2.ipynb) - Aggregationen, Joins & Reshaping
    - [Teil 3](https://colab.research.google.com/github/bettercodepaul/data2day_2023_polars/blob/main/data2day_2023_Polars_Teil_3.ipynb) - Custom Expressions, Lazy Mode und Streaming

## Weiterführende Ressourcen

* Dash Gallery (Umfangreiche Beispielsammlung für Dash-Apps, inkl. Source-Code): https://dash-gallery.plotly.host/Portal/
* lokale Python-Umgebung: uv (https://docs.astral.sh/uv/)
* lokale Python-IDE: VSCode (https://code.visualstudio.com/)
* Zugriff auf Datenbanken: SQLAlchemy (https://www.sqlalchemy.org/)
